In [13]:
pip install diffusers transformers accelerate bitsandbytes wandb pycocotools kagglehub

Note: you may need to restart the kernel to use updated packages.


In [14]:
%%writefile train_multi_controlnet1.py
import os
import torch
import numpy as np
import random
import shutil
import requests
import zipfile
from tqdm.auto import tqdm
from PIL import Image, ImageDraw
from pycocotools.coco import COCO
from torchvision import transforms
import bitsandbytes as bnb
import wandb

# Accelerate & Diffusers
from accelerate import Accelerator
from diffusers import (
    StableDiffusionControlNetPipeline, 
    ControlNetModel, 
    MultiControlNetModel,
    DDPMScheduler,
    AutoencoderKL,
    UNet2DConditionModel
)
from transformers import AutoTokenizer, CLIPTextModel

# Enable TF32 for H100 speedup
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# --- CONFIGURATION ---
class Config:
    # Paths
    DATA_ROOT = "data"
    COCO_ROOT = os.path.join(DATA_ROOT, "coco2017")
    TRAIN_IMG_DIR = os.path.join(COCO_ROOT, "train2017")
    TRAIN_ANN_FILE = os.path.join(COCO_ROOT, "annotations/instances_train2017.json")
    
    MODEL_ID = "runwayml/stable-diffusion-v1-5"
    OUTPUT_DIR = "controlnet-coco-multi-h100-new"
    
    # Optional: Path to previously trained single ControlNets to jumpstart training
    PRETRAINED_SEG_PATH = None 
    PRETRAINED_BBOX_PATH = None
    
    # Checkpoints
    RESUME_FROM_CHECKPOINT = "latest"
    
    # Hyperparameters
    RESOLUTION = 512
    BATCH_SIZE = 64
    GRAD_ACCUM_STEPS = 1
    LEARNING_RATE = 2e-5   
    NUM_EPOCHS = 15        
    
    # Logging
    LOG_INTERVAL = 200       
    LOG_BATCH_SIZE = 8
    SAVE_INTERVAL = 1000
    MAX_SAMPLES = None
    
    # Data Loading
    NUM_WORKERS = 16
    
    # Dropout Probabilities
    PROMPT_DROPOUT_PROB = 0.4
    PROB_SEG_ONLY = 0.35
    PROB_BBOX_ONLY = 0.35

# --- DOWNLOAD UTILS ---
def download_file(url, save_path):
    if os.path.exists(save_path): return
    print(f"Downloading {url} to {save_path}...")
    response = requests.get(url, stream=True)
    with open(save_path, "wb") as file:
        for data in response.iter_content(1024):
            file.write(data)

def unzip_file(zip_path, extract_to):
    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def setup_coco_dataset():
    os.makedirs(Config.DATA_ROOT, exist_ok=True)
    os.makedirs(Config.COCO_ROOT, exist_ok=True)
    
    train_images_url = "http://images.cocodataset.org/zips/train2017.zip"
    annotations_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
    
    zip_train = os.path.join(Config.DATA_ROOT, "train2017.zip")
    zip_ann = os.path.join(Config.DATA_ROOT, "annotations.zip")
    
    if not os.path.exists(Config.TRAIN_IMG_DIR):
        download_file(train_images_url, zip_train)
        unzip_file(zip_train, Config.COCO_ROOT)
        if os.path.exists(zip_train): os.remove(zip_train)
        
    if not os.path.exists(os.path.dirname(Config.TRAIN_ANN_FILE)):
        download_file(annotations_url, zip_ann)
        unzip_file(zip_ann, Config.COCO_ROOT)
        if os.path.exists(zip_ann): os.remove(zip_ann)

# --- DATASET CLASS ---
class COCOMultiDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, ann_file, tokenizer, size=512, max_samples=None):
        self.img_dir = img_dir
        self.coco = COCO(ann_file)
        self.img_ids = self.coco.getImgIds()
        self.tokenizer = tokenizer
        self.size = size
        self.img_ids = [img_id for img_id in self.img_ids if len(self.coco.getAnnIds(imgIds=img_id)) > 0]
        if max_samples: self.img_ids = self.img_ids[:max_samples]

        self.image_transforms = transforms.Compose([
            transforms.Resize(size, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]), 
        ])
        
        self.cond_transforms = transforms.Compose([
            transforms.Resize(size, interpolation=transforms.InterpolationMode.NEAREST),
            transforms.CenterCrop(size),
            transforms.ToTensor(), 
        ])
        
        self.color_map = self._generate_color_map()

    def _generate_color_map(self):
        cats = self.coco.loadCats(self.coco.getCatIds())
        palette = {}
        for cat in cats:
            import hashlib
            h = hashlib.md5(str(cat['id']).encode()).hexdigest()
            palette[cat['id']] = (int(h[0:2], 16), int(h[2:4], 16), int(h[4:6], 16))
        return palette

    def draw_segmentation_map(self, img_shape, anns):
        mask = np.zeros((img_shape[1], img_shape[0], 3), dtype=np.uint8)
        anns = sorted(anns, key=lambda x: x['area'], reverse=True)
        for ann in anns:
            color = self.color_map.get(ann['category_id'], (255, 255, 255))
            binary_mask = self.coco.annToMask(ann)
            mask[binary_mask == 1] = color
        return Image.fromarray(mask)

    def draw_bbox_map(self, img_shape, anns):
        mask = np.zeros((img_shape[1], img_shape[0], 3), dtype=np.uint8)
        canvas = Image.fromarray(mask)
        draw = ImageDraw.Draw(canvas)
        anns = sorted(anns, key=lambda x: x['area'], reverse=True)
        for ann in anns:
            x, y, w, h = ann['bbox']
            color = self.color_map.get(ann['category_id'], (255, 255, 255))
            draw.rectangle([x, y, x+w, y+h], fill=color, outline=None)
        return canvas

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        try:
            image_path = os.path.join(self.img_dir, img_info['file_name'])
            image = Image.open(image_path).convert("RGB")
        except Exception:
            return self.__getitem__((idx + 1) % len(self))
            
        anns = self.coco.loadAnns(self.coco.getAnnIds(imgIds=img_id))
        seg_image = self.draw_segmentation_map(image.size, anns)
        bbox_image = self.draw_bbox_map(image.size, anns)
        
        cat_ids = [ann['category_id'] for ann in anns]
        cat_names = list(set([cat['name'] for cat in self.coco.loadCats(cat_ids)]))
        text_prompt = "" if random.random() < Config.PROMPT_DROPOUT_PROB else \
                      f"A photorealistic image containing {', '.join(cat_names)}" if cat_names else "A photorealistic image"
        
        return {
            "pixel_values": self.image_transforms(image),
            "seg_pixel_values": self.cond_transforms(seg_image),
            "bbox_pixel_values": self.cond_transforms(bbox_image),
            "input_ids": self.tokenizer(
                text_prompt, max_length=self.tokenizer.model_max_length, 
                padding="max_length", truncation=True, return_tensors="pt"
            ).input_ids[0],
            "raw_prompt": text_prompt
        }

# --- VALIDATION HELPER ---
def log_validation(accelerator, controlnet_seg, controlnet_bbox, unet, vae, text_encoder, tokenizer, val_batch, step):
    if not accelerator.is_main_process: return
    print(f"Running Validation at step {step}...")

    try:
        multi_controlnet = MultiControlNetModel([controlnet_seg, controlnet_bbox])
        pipeline = StableDiffusionControlNetPipeline.from_pretrained(
            Config.MODEL_ID,
            vae=vae, text_encoder=text_encoder, tokenizer=tokenizer,
            unet=unet, controlnet=multi_controlnet,
            safety_checker=None, torch_dtype=torch.bfloat16
        ).to(accelerator.device)
        pipeline.set_progress_bar_config(disable=True)
        
        log_images = []
        num_samples = min(len(val_batch["raw_prompt"]), Config.LOG_BATCH_SIZE)
        
        def to_pil(tensor):
            tensor = tensor.detach().cpu().float()
            if tensor.shape[0] == 3: tensor = tensor.permute(1, 2, 0)
            if tensor.min() < 0: tensor = (tensor + 1) / 2.0
            tensor = tensor.clamp(0, 1).numpy()
            return Image.fromarray((tensor * 255).astype(np.uint8))

        for i in range(num_samples):
            prompt = val_batch["raw_prompt"][i] or "A photorealistic image"
            gt_image = to_pil(val_batch["pixel_values"][i])
            seg_image = to_pil(val_batch["seg_pixel_values"][i])
            bbox_image = to_pil(val_batch["bbox_pixel_values"][i])
            
            generator = torch.Generator(device=accelerator.device).manual_seed(42 + i)
            with torch.autocast("cuda", dtype=torch.bfloat16):
                pred_both = pipeline(prompt, image=[seg_image, bbox_image], num_inference_steps=20, generator=generator, controlnet_conditioning_scale=[1.0, 1.0]).images[0]
                pred_seg = pipeline(prompt, image=[seg_image, bbox_image], num_inference_steps=20, generator=generator, controlnet_conditioning_scale=[1.0, 0.0]).images[0]
                pred_bbox = pipeline(prompt, image=[seg_image, bbox_image], num_inference_steps=20, generator=generator, controlnet_conditioning_scale=[0.0, 1.0]).images[0]

            log_images.extend([
                wandb.Image(seg_image, caption=f"{i} Seg In"),
                wandb.Image(bbox_image, caption=f"{i} BBox In"),
                wandb.Image(gt_image, caption=f"{i} Truth"),
                wandb.Image(pred_both, caption=f"{i} Both"),
                wandb.Image(pred_seg, caption=f"{i} Seg Only"),
                wandb.Image(pred_bbox, caption=f"{i} BBox Only")
            ])
        
        tracker = accelerator.get_tracker("wandb")
        if tracker: tracker.log({"validation": log_images}, step=step)
        
        del pipeline, multi_controlnet
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"Skipping validation log due to error: {e}")

# --- MAIN FUNCTION ---
def main():
    setup_coco_dataset()

    accelerator = Accelerator(
        gradient_accumulation_steps=Config.GRAD_ACCUM_STEPS,
        mixed_precision="bf16",
        log_with="wandb",
    )
    
    if accelerator.is_main_process:
        wandb_key = os.getenv("wandb")
        if wandb_key: wandb.login(key=wandb_key)
        cfg_dict = {k: v for k, v in Config.__dict__.items() if not k.startswith("__")}
        accelerator.init_trackers("controlnet-coco-multi-h100", config=cfg_dict)

    tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_ID, subfolder="tokenizer", use_fast=False)
    noise_scheduler = DDPMScheduler.from_pretrained(Config.MODEL_ID, subfolder="scheduler")
    text_encoder = CLIPTextModel.from_pretrained(Config.MODEL_ID, subfolder="text_encoder", torch_dtype=torch.bfloat16)
    vae = AutoencoderKL.from_pretrained(Config.MODEL_ID, subfolder="vae", torch_dtype=torch.bfloat16)
    unet = UNet2DConditionModel.from_pretrained(Config.MODEL_ID, subfolder="unet", torch_dtype=torch.bfloat16)
    
    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)
    unet.requires_grad_(False)
    
    # Initialize Models
    if Config.PRETRAINED_SEG_PATH:
        controlnet_seg = ControlNetModel.from_pretrained(Config.PRETRAINED_SEG_PATH)
    else:
        controlnet_seg = ControlNetModel.from_unet(unet)

    if Config.PRETRAINED_BBOX_PATH:
        controlnet_bbox = ControlNetModel.from_pretrained(Config.PRETRAINED_BBOX_PATH)
    else:
        controlnet_bbox = ControlNetModel.from_unet(unet)

    controlnet_seg.train()
    controlnet_bbox.train()
    controlnet_seg.enable_gradient_checkpointing()
    controlnet_bbox.enable_gradient_checkpointing()
    unet.enable_gradient_checkpointing()

    dataset = COCOMultiDataset(Config.TRAIN_IMG_DIR, Config.TRAIN_ANN_FILE, tokenizer, max_samples=Config.MAX_SAMPLES)
    train_dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=Config.BATCH_SIZE, shuffle=True,
        num_workers=Config.NUM_WORKERS, pin_memory=True, drop_last=True
    )

    val_batch = next(iter(train_dataloader))
    params = list(controlnet_seg.parameters()) + list(controlnet_bbox.parameters())
    optimizer = bnb.optim.AdamW8bit(params, lr=Config.LEARNING_RATE)

    controlnet_seg, controlnet_bbox, optimizer, train_dataloader = accelerator.prepare(
        controlnet_seg, controlnet_bbox, optimizer, train_dataloader
    )
    
    vae.to(accelerator.device)
    text_encoder.to(accelerator.device)
    unet.to(accelerator.device)

    global_step = 0
    start_epoch = 0
    
    if Config.RESUME_FROM_CHECKPOINT == "latest" and os.path.exists(Config.OUTPUT_DIR):
        dirs = [d for d in os.listdir(Config.OUTPUT_DIR) if d.startswith("checkpoint")]
        if dirs:
            path = sorted(dirs, key=lambda x: int(x.split("-")[1]))[-1]
            accelerator.print(f"Resuming from {path}")
            accelerator.load_state(os.path.join(Config.OUTPUT_DIR, path))
            global_step = int(path.split("-")[1])
            start_epoch = global_step // len(train_dataloader)

    if accelerator.is_main_process: print(f"Starting training from Step {global_step}...")
    
    for epoch in range(start_epoch, Config.NUM_EPOCHS):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process, desc=f"Epoch {epoch}")
        
        for batch in train_dataloader:
            with accelerator.accumulate([controlnet_seg, controlnet_bbox]):
                latents = vae.encode(batch["pixel_values"].to(dtype=torch.bfloat16)).latent_dist.sample() * vae.config.scaling_factor
                noise = torch.randn_like(latents)
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=latents.device).long()
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
                
                encoder_hidden_states = text_encoder(batch["input_ids"])[0]
                
                # --- SIMPLIFIED LOGIC ---
                r = random.random()
                
                # Keep track of active residuals in lists
                active_down = []
                active_mid = []

                # 1. SEGMENTATION
                # Logic: Run if we are in "Seg Only" (<0.35) OR "Both" (>0.7)
                if r < Config.PROB_SEG_ONLY or r >= (Config.PROB_SEG_ONLY + Config.PROB_BBOX_ONLY):
                    real_seg = batch["seg_pixel_values"].to(dtype=torch.bfloat16)
                    d, m = controlnet_seg(
                        noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states, 
                        controlnet_cond=real_seg, return_dict=False
                    )
                    active_down.append(d)
                    active_mid.append(m)

                # 2. BBOX
                # Logic: Run if we are in "BBox Only" (0.35 - 0.7) OR "Both" (>0.7)
                if (r >= Config.PROB_SEG_ONLY and r < (Config.PROB_SEG_ONLY + Config.PROB_BBOX_ONLY)) or r >= (Config.PROB_SEG_ONLY + Config.PROB_BBOX_ONLY):
                    real_bbox = batch["bbox_pixel_values"].to(dtype=torch.bfloat16)
                    d, m = controlnet_bbox(
                        noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states, 
                        controlnet_cond=real_bbox, return_dict=False
                    )
                    active_down.append(d)
                    active_mid.append(m)

                # --- AGGREGATION (NO "IF TENSOR" CHECKS) ---
                if len(active_down) == 0:
                    # Both dropped (shouldn't happen with current probs, but safe)
                    down_block_res = None
                    mid_block_res = None
                else:
                    # Sum up whatever is in the list
                    down_block_res = active_down[0]
                    mid_block_res = active_mid[0]
                    
                    for i in range(1, len(active_down)):
                        down_block_res = [a + b for a, b in zip(down_block_res, active_down[i])]
                        mid_block_res = mid_block_res + active_mid[i]
                
                # CAST TO BF16
                if down_block_res is not None:
                    down_block_res = [res.to(dtype=torch.bfloat16) for res in down_block_res]
                    mid_block_res = mid_block_res.to(dtype=torch.bfloat16)

                # UNET FORWARD
                model_pred = unet(
                    noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states,
                    down_block_additional_residuals=down_block_res,
                    mid_block_additional_residual=mid_block_res,
                ).sample
                
                loss = torch.nn.functional.mse_loss(model_pred.float(), noise.float(), reduction="mean")
                accelerator.backward(loss)
                optimizer.step()
                optimizer.zero_grad()

            if accelerator.sync_gradients:
                global_step += 1
                progress_bar.update(1)
                
                if accelerator.is_main_process:
                    accelerator.log({"train_loss": loss.item()}, step=global_step)
                    if global_step % Config.LOG_INTERVAL == 0:
                        u_seg = accelerator.unwrap_model(controlnet_seg)
                        u_bbox = accelerator.unwrap_model(controlnet_bbox)
                        log_validation(accelerator, u_seg, u_bbox, unet, vae, text_encoder, tokenizer, val_batch, global_step)
                    if global_step % Config.SAVE_INTERVAL == 0:
                        save_path = os.path.join(Config.OUTPUT_DIR, f"checkpoint-{global_step}")
                        accelerator.save_state(save_path)
                        try:
                            checkpoints = sorted([d for d in os.listdir(Config.OUTPUT_DIR) if d.startswith("checkpoint-")], key=lambda x: int(x.split("-")[1]))
                            if len(checkpoints) > 2:
                                for ckpt in checkpoints[:-2]: shutil.rmtree(os.path.join(Config.OUTPUT_DIR, ckpt))
                        except Exception: pass
    
    if accelerator.is_main_process:
        accelerator.unwrap_model(controlnet_seg).save_pretrained(os.path.join(Config.OUTPUT_DIR, "final_seg"))
        accelerator.unwrap_model(controlnet_bbox).save_pretrained(os.path.join(Config.OUTPUT_DIR, "final_bbox"))
        accelerator.end_training()

if __name__ == "__main__":
    main()

Overwriting train_multi_controlnet1.py


In [15]:
!accelerate launch --mixed_precision=bf16 train_multi_controlnet1.py

^C
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/bin/accelerate", line 3, in <module>
    from accelerate.commands.accelerate_cli import main
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/accelerate/__init__.py", line 16, in <module>
    from .accelerator import Accelerator
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/accelerate/accelerator.py", line 31, in <module>
    import torch
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/__init__.py", line 416, in <module>
    from torch._C import *  # noqa: F403
    ^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 463, in _lock_unlock_module
KeyboardInterrupt


In [ ]:
!pip install huggingface_hub

In [16]:
%%writefile upload_hf.py

import os
import shutil
import torch
from huggingface_hub import HfApi, create_repo
from diffusers import ControlNetModel, UNet2DConditionModel

# --- CONFIGURATION ---
HF_USERNAME = "ritishshrirao"
REPO_NAME = "controlnet-coco-multi"
MODEL_ID = f"{HF_USERNAME}/{REPO_NAME}"

# Path where your training checkpoints are
LOCAL_OUTPUT_DIR = "/teamspace/studios/this_studio/controlnet-coco-multi-h100-new"
BASE_MODEL_ID = "runwayml/stable-diffusion-v1-5"

def get_latest_checkpoint(base_path):
    if not os.path.exists(base_path): return None
    checkpoints = [d for d in os.listdir(base_path) if d.startswith("checkpoint-")]
    if not checkpoints: return None
    # Sort by integer step
    checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))
    return os.path.join(base_path, checkpoints[-1])

def load_weights_to_model(model, bin_path):
    print(f"Loading weights from {bin_path}...")
    # Load state dict (map_location='cpu' to save VRAM)
    state_dict = torch.load(bin_path, map_location="cpu")
    
    # Fix keys if they were saved with 'module.' prefix (DistributedDataParallel)
    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith("module."):
            new_state_dict[k[7:]] = v
        else:
            new_state_dict[k] = v
            
    # Load into model
    missing, unexpected = model.load_state_dict(new_state_dict, strict=False)
    if len(missing) > 0:
        print(f"⚠️ Warning: Missing keys: {len(missing)}")
    print("Weights loaded successfully.")
    return model

def extract_and_upload():
    # 1. Setup Auth
    hf_token = os.getenv("HF")
    if not hf_token:
        print("❌ Error: 'HF' environment variable not found.")
        return

    # 2. Find Checkpoint
    latest_ckpt_path = get_latest_checkpoint(LOCAL_OUTPUT_DIR)
    if not latest_ckpt_path:
        print("❌ No checkpoints found.")
        return
    print(f"📂 Processing Checkpoint: {latest_ckpt_path}")

    # 3. Identify Weight Files
    # In accelerate, the order passed to prepare() determines the index.
    # Script order: prepare(controlnet_seg, controlnet_bbox, ...)
    seg_weights_path = os.path.join(latest_ckpt_path, "pytorch_model_0.bin")
    bbox_weights_path = os.path.join(latest_ckpt_path, "pytorch_model_1.bin")

    if not os.path.exists(seg_weights_path) or not os.path.exists(bbox_weights_path):
        print(f"❌ Could not find model bins in {latest_ckpt_path}. Expected pytorch_model_0.bin and pytorch_model_1.bin")
        return

    # 4. Initialize Clean Models
    print("🏗️  Initializing model architecture...")
    unet = UNet2DConditionModel.from_pretrained(BASE_MODEL_ID, subfolder="unet")
    
    # Create empty ControlNets
    c_seg = ControlNetModel.from_unet(unet)
    c_bbox = ControlNetModel.from_unet(unet)
    
    # 5. Load Weights
    c_seg = load_weights_to_model(c_seg, seg_weights_path)
    c_bbox = load_weights_to_model(c_bbox, bbox_weights_path)

    # 6. Save Clean Versions (Safetensors)
    export_dir = "temp_clean_models"
    if os.path.exists(export_dir): shutil.rmtree(export_dir)
    os.makedirs(export_dir, exist_ok=True)

    print("💾 Saving clean safetensors models...")
    c_seg.save_pretrained(os.path.join(export_dir, "segmentation"), safe_serialization=True)
    c_bbox.save_pretrained(os.path.join(export_dir, "bbox"), safe_serialization=True)

    # 7. Upload to Hugging Face
    api = HfApi(token=hf_token)
    
    print(f"🌐 Creating/Checking Repo: {MODEL_ID}")
    try:
        create_repo(repo_id=MODEL_ID, exist_ok=True, private=False, token=hf_token)
    except Exception as e:
        print(f"Repo info: {e}")

    print("🚀 Uploading to Hugging Face Hub...")
    try:
        # Upload Segmentation
        print("... Uploading Segmentation ControlNet")
        api.upload_folder(
            folder_path=os.path.join(export_dir, "segmentation"),
            repo_id=MODEL_ID,
            path_in_repo="segmentation",
            commit_message="Upload clean segmentation weights"
        )
        
        # Upload BBox
        print("... Uploading BBox ControlNet")
        api.upload_folder(
            folder_path=os.path.join(export_dir, "bbox"),
            repo_id=MODEL_ID,
            path_in_repo="bbox",
            commit_message="Upload clean bbox weights"
        )
        
        print("\n✅ Success! Models available at:")
        print(f"👉 https://huggingface.co/{MODEL_ID}/tree/main")
        
    except Exception as e:
        print(f"❌ Upload Failed: {e}")
    finally:
        # Cleanup temp folder
        if os.path.exists(export_dir):
            shutil.rmtree(export_dir)

if __name__ == "__main__":
    extract_and_upload()

Writing upload_hf.py


In [1]:
!python upload_hf.py

📂 Processing Checkpoint: /teamspace/studios/this_studio/controlnet-coco-multi-h100-new/checkpoint-3000
❌ Could not find model bins in /teamspace/studios/this_studio/controlnet-coco-multi-h100-new/checkpoint-3000. Expected pytorch_model_0.bin and pytorch_model_1.bin


In [2]:
!ls /teamspace/studios/this_studio/controlnet-coco-multi-h100-new/checkpoint-3000

model.safetensors  model_1.safetensors	optimizer.bin  random_states_0.pkl


In [17]:
!pip install huggingface_hub